In [14]:
from magent2.environments import battle_v4
import os
import cv2
from final_pretrained_agent import BOSS_DQN
from pretrained_agent import NOOB_DQN
from self_trained_agent import DoubleQPolicy, DuelingQPolicy
import torch

In [15]:
env = battle_v4.env(map_size=45, render_mode="rgb_array", max_cycles=300)

In [16]:
vid_dir = "video"
os.makedirs(vid_dir, exist_ok=True)
fps = 35
frames = []

In [17]:
boss_red = BOSS_DQN()
self_blue = DoubleQPolicy("solution\DeepQNetwork\Double Q\\trained_pth\\blue_deepQ.pth")

In [18]:
test_input = torch.rand(13, 13, 5)
boss_red.get_action(test_input)
self_blue.get_action(test_input)

18

In [19]:
frames = []
env.reset()

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        agent_handle = agent.split("_")[0]
        observation = torch.tensor(observation)
        if agent_handle == "red":
            with torch.no_grad():
                action = boss_red.get_action(observation)
        else:
            with torch.no_grad():
                action = self_blue.get_action(observation)
    env.step(action)

    if env.agents and agent == env.agents[-1]:
        frames.append(env.render())

height, width, _ = frames[0].shape
out = cv2.VideoWriter(
    os.path.join(vid_dir, "battle.mp4"),
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (width, height),
)

for frame in frames:
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    out.write(frame_bgr)
out.release()
env.close()